## Custom API +  Custom Worker

#### Import dependencies

In [ ]:
# syft absolute
import syft as sy
from syft.service.settings.settings import NodeSettingsUpdate
from syft.service.worker.worker_image import SyftWorkerImage

In [ ]:
## remote mode
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
domain_client = sy.login(
    email="info@openmined.org",
    url="http://127.0.0.1",
    password="",
)

In [ ]:
# # python mode
# # !uv pip install google-cloud-bigquery db_dtypes
# node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True, reset=True)
# domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_client.worker_pools

## Register a custom Image

In [ ]:
registry = "us-central1-docker.pkg.dev/reddit-testing-415005/syft-registry-us"
backend_version = None
assert backend_version is not None

custom_dockerfile_str = f"""
FROM {registry}/openmined/grid-backend:{backend_version}

RUN pip install google-cloud-bigquery[all]==3.20.1 db-dtypes==1.2.0

""".strip()
print(custom_dockerfile_str)

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=custom_dockerfile_str)

In [ ]:
submit_result = domain_client.api.services.worker_image.submit(
    worker_config=docker_config
)
submit_result

In [ ]:
dockerfile_list = domain_client.images.get_all()
dockerfile_list

In [ ]:
workerimage = next(
    (
        image
        for image in dockerfile_list
        if not image.is_prebuilt and image.config.dockerfile == custom_dockerfile_str
    ),
    None,
)

assert isinstance(workerimage, SyftWorkerImage), str(workerimage)
workerimage

In [ ]:
external_registry = registry
worker_docker_tag = "openmined/bigquery:0.0.1"

In [ ]:
registry_add_result = domain_client.api.services.image_registry.add(external_registry)
registry_add_result

In [ ]:
registries = domain_client.api.services.image_registry.get_all()
registry_uid = next((r.id for r in registries if r.url == external_registry), None)
registry_uid

In [ ]:
# build with registry_uid
docker_build_result = domain_client.api.services.worker_image.build(
    image_uid=workerimage.id,
    tag=worker_docker_tag,
    registry_uid=registry_uid,
)
print(docker_build_result)

In [ ]:
image_list = domain_client.images.get_all()
# we can also index with string using the repo_with_tag format
workerimage = next((image for image in image_list if image.id == workerimage.id), None)
assert workerimage.is_built is True
workerimage

In [ ]:
push_result = domain_client.api.services.worker_image.push(workerimage.id)
push_result

In [ ]:
worker_pool_name = "bigquery-pool"
domain_client.api.services.worker_pool.launch(
    pool_name=worker_pool_name,
    image_uid=workerimage.id,
    num_workers=1,
)

In [ ]:
domain_client.worker_pools[1]

In [ ]:
new_default_worker_pool = NodeSettingsUpdate(default_worker_pool=worker_pool_name)
domain_client.settings.update(settings=new_default_worker_pool)

In [ ]:
SERVICE_ACCOUNT = {}

In [ ]:
# debug manually
# from google.oauth2 import service_account
# from google.cloud import bigquery
# credentials = service_account.Credentials.from_service_account_info(SERVICE_ACCOUNT)
# scoped_credentials = credentials.with_scopes(['https://www.googleapis.com/auth/bigquery'])
# scoped_credentials = credentials.with_scopes(['https://www.googleapis.com/auth/cloud-platform'])

# client = bigquery.Client(
#     credentials=scoped_credentials,
#     location="us-west1",
# )
# sql="SELECT * FROM reddit-testing-415005.test_1gb.accounts limit 10"
# rows = client.query_and_wait(
#     sql
# )
# g = sy.ActionObject.from_obj(rows)

In [ ]:
@sy.api_endpoint_method(
    settings={"credentials": SERVICE_ACCOUNT, "project_id": "reddit-testing-415005"}
)
def public_function(context, sql: str) -> str:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    credentials = service_account.Credentials.from_service_account_info(
        context.settings["credentials"]
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/bigquery"]
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )

    rows = client.query_and_wait(
        sql,
        project=context.settings["project_id"],
    )

    return rows


@sy.api_endpoint_method(
    settings={"credentials": SERVICE_ACCOUNT, "project_id": "reddit-testing-415005"}
)
def private_function(context, sql: str) -> str:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    credentials = service_account.Credentials.from_service_account_info(
        context.settings["credentials"]
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/bigquery"]
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )

    rows = client.query_and_wait(
        sql,
        project=context.settings["project_id"],
    )

    return rows

In [ ]:
new_endpoint = sy.TwinAPIEndpoint(
    path="bigquery.query",
    mock_function=public_function,
    private_function=private_function,
    description="Lore ipsulum ...",
)

In [ ]:
response = domain_client.api.services.api.delete(endpoint_path="bigquery.query")
response

In [ ]:
response = domain_client.api.services.api.add(endpoint=new_endpoint)
response

In [ ]:
domain_client.refresh()

In [ ]:
@sy.syft_function_single_use(
    endpoint=domain_client.api.services.bigquery.query,
    worker_pool_name=worker_pool_name,
)
def job_function(endpoint):
    result = endpoint(
        sql="SELECT * FROM reddit-testing-415005.test_1gb.accounts limit 10"
    )
    return result

In [ ]:
new_project = sy.Project(
    name="My Cool UN Project",
    description="Hi, I want to calculate the trade volume in million's with my cool code.",
    members=[domain_client],
)

In [ ]:
result = new_project.create_code_request(job_function, domain_client)
domain_client.requests[-1].approve()

In [ ]:
domain_client.settings.get().default_worker_pool

In [ ]:
job = domain_client.code.job_function(
    endpoint=domain_client.api.services.bigquery.query, blocking=False
)

In [ ]:
job

In [ ]:
domain_client.jobs